In [6]:
#分箱操作
import pandas as pd
data = pd.read_csv('data2.csv',index_col=0,header=0)

In [7]:
data.shape

(10000, 10)

'''特征变量选择(排序)对于数据分析、机器学习从业者来说非常重要。
好的特征选择能够提升模型的性能，更能帮助我们理解数据的特点、底层结构，
这对进一步改善模型、算法都有着重要作用。至于Python的变量选择代码实现可以参考结合Scikit-learn介绍几种常用的特征选择方法。

在本文中，我们采用信用评分模型的变量选择方法，
通过WOE分析方法，即是通过比较指标分箱和对应分箱的违约概率来确定指标是否符合经济意义。
首先我们对变量进行离散化（分箱）处理。

'''
#https://www.jianshu.com/p/f931a4df202c


In [8]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import scipy.stats.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math

# 定义自动分箱函数
def mono_bin(Y, X, n = 20):
    r = 0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n)})
        d2 = d1.groupby('Bucket', as_index = True)
        r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min']=d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d3['goodattribute']=d3['sum']/good
    d3['badattribute']=(d3['total']-d3['sum'])/bad
    iv=((d3['goodattribute']-d3['badattribute'])*d3['woe']).sum()
    d4 = (d3.sort_index(by = 'min'))
    print("=" * 60)
    print(d4)
    cut=[]
    cut.append(float('-inf'))
    for i in range(1,n+1):
        qua=X.quantile(i/(n+1))
        cut.append(round(qua,4))
    cut.append(float('inf'))
    woe=list(d4['woe'].round(3))
    return d4,iv,cut,woe
#自定义分箱函数
def self_bin(Y,X,cat):
    good=Y.sum()
    bad=Y.count()-good
    d1=pd.DataFrame({'X':X,'Y':Y,'Bucket':pd.cut(X,cat)})
    d2=d1.groupby('Bucket', as_index = True)
    d3 = pd.DataFrame(d2.X.min(), columns=['min'])
    d3['min'] = d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe'] = np.log((d3['rate'] / (1 - d3['rate'])) / (good / bad))
    d3['goodattribute'] = d3['sum'] / good
    d3['badattribute'] = (d3['total'] - d3['sum']) / bad
    iv = ((d3['goodattribute'] - d3['badattribute']) * d3['woe']).sum()
    d4 = (d3.sort_index(by='min'))
    print("=" * 60)
    print(d4)
    woe = list(d4['woe'].round(3))
    return d4, iv,woe
#用woe代替
def replace_woe(series,cut,woe):
    list=[]
    i=0
    while i<len(series):
        value=series[i]
        j=len(cut)-2
        m=len(cut)-2
        while j>=0:
            if value>=cut[j]:
                j=-1
            else:
                j -=1
                m -= 1
        list.append(woe[m])
        i += 1
    return list
#计算分数函数
def get_score(coe,woe,factor):
    scores=[]
    for w in woe:
        score=round(coe*w*factor,0)
        scores.append(score)
    return scores

#根据变量计算分数
def compute_score(series,cut,score):
    list = []
    i = 0
    while i < len(series):
        value = series[i]
        j = len(cut) - 2
        m = len(cut) - 2
        while j >= 0:
            if value >= cut[j]:
                j = -1
            else:
                j -= 1
                m -= 1
        list.append(score[m])
        i += 1
    return list


In [9]:
    pinf = float('inf')#正无穷大
    ninf = float('-inf')#负无穷大

In [19]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
y,10000.0,0.199500,0.399644,0.0,0.0,0.0,0.0000,1.00
six_cycle_mp_num,10000.0,0.354800,0.873037,0.0,0.0,0.0,0.0000,5.00
six_cycle_mp_avg_amt,10000.0,2227.685166,8820.798447,0.0,0.0,0.0,0.0000,265792.00
epp_nbr_12m,10000.0,1.226500,3.057373,0.0,0.0,0.0,1.0000,102.00
epp_avg_amt_12m,10000.0,810.992281,2380.266311,0.0,0.0,0.0,613.5225,48123.42
alop_nbr_12m,10000.0,0.318700,0.980115,0.0,0.0,0.0,0.0000,24.00
alop_avg_amt_12m,10000.0,602.073626,2431.722290,0.0,0.0,0.0,0.0000,50833.33
six_bill_low_repay_num,10000.0,0.978100,1.602647,0.0,0.0,0.0,1.0000,6.00
consume_amt_session,10000.0,9981.344644,17264.501651,0.0,960.0,4000.0,12444.1975,492018.00
consume_num_session,10000.0,8.534800,17.485828,0.0,1.0,3.0,8.0000,271.00


In [ ]:
先做自动最优分箱

In [21]:
dfx1,ivx1,cutx1,woex1=mono_bin(data.y, data.six_cycle_mp_num, n=20)

ValueError: Bin edges must be unique: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  1.,  2.,  5.])

In [26]:
#dfx1,ivx1,cutx1,woex1=mono_bin(data.y, data.six_cycle_mp_num, n=5)#
#dfx2,ivx2,cutx2,woex2=mono_bin(data.y, data.six_cycle_mp_avg_amt, n=5)#
#dfx3,ivx3,cutx3,woex3 =mono_bin(data.y, data.epp_nbr_12m, n=5)#
#dfx4,ivx4,cutx4,woex4 =mono_bin(data.y, data.epp_avg_amt_12m, n=6)#
#dfx5,ivx5,cutx5,woex5 =mono_bin(data.y, data.alop_nbr_12m, n=6)#
#dfx6,ivx6,cutx6,woex6 =mono_bin(data.y, data.alop_avg_amt_12m, n=6)#
#dfx7,ivx7,cutx7,woex7 =mono_bin(data.y, data.six_bill_low_repay_num, n=6)#
dfx8,ivx8,cutx8,woex8 =mono_bin(data.y, data.consume_amt_session, n=6)#
#dfx9,ivx9,cutx9,woex9 =mono_bin(data.y, data.consume_num_session, n=6)#

             min       max     sum  total    rate           woe  \
Bucket                                                            
[0, 492018]  0.0  492018.0  1995.0  10000  0.1995  2.220446e-16   

             goodattribute  badattribute  
Bucket                                    
[0, 492018]            1.0           1.0  


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:29: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [ ]:
#dfx2,ivx2cutx2,woex2=mono_bin(data.y, data.age, n=3)#age-年龄
dfx3,ivx3,cutx3,woex3 =mono_bin(data.y, data.xaccount_age, n=5)#xaccount_age-账龄
dfx4,ivx4,cutx4,woex4 =mono_bin(data.y, data.cred_limit, n=6)#cred_limit-行用卡额度
dfx5,ivx5,cutx5,woex5 =mono_bin(data.y, data.this_bill_rate, n=6)#this_bill_rate-行用卡额度
dfx6,ivx6,cutx6,woex6 =mono_bin(data.y, data.this_bill_mp, n=6)#this_bill_mp-行用卡额度
#dfx7,ivx7,cutx7,woex7 =mono_bin(data.y, data.last_mp_days, n=6)#last_mp_days-行用卡额度
#dfx8,ivx8,cutx8,woex8 =mono_bin(data.y, data.six_bill_num, n=6)#six_bill_num-行用卡额度
dfx9,ivx9,cutx9,woex9 =mono_bin(data.y, data.six_bill_avg_amt, n=6)#six_bill_avg_amt-行用卡额度
#dfx10,ivx10,cutx10,woex10 =mono_bin(data.y, data.six_cycle_mp_num, n=6)#six_cycle_mp_num-行用卡额度
#dfx11,ivx11,cutx11,woex11 =mono_bin(data.y, data.six_cycle_mp_avg_amt, n=6)#six_cycle_mp_avg_amt-行用卡额度
dfx12,ivx12,cutx12,woex12 =mono_bin(data.y, data.six_bill_avg_debt_rate, n=6)#six_bill_avg_debt_rate-行用卡额度
#dfx13,ivx13,cutx13,woex13 =mono_bin(data.y, data.six_bill_low_repay_num, n=6)#six_bill_low_repay_num-行用卡额度
#dfx14,ivx14,cutx14,woex14 =mono_bin(data.y, data.epp_nbr_12m, n=6)#epp_nbr_12m-行用卡额度
#dfx15,ivx15,cutx15,woex15 =mono_bin(data.y, data.alop_nbr_12m, n=6)#alop_nbr_12m-行用卡额度
#dfx16,ivx16,cutx16,woex16 =mono_bin(data.y, data.epp_avg_amt_12m, n=6)#epp_avg_amt_12m-行用卡额度
#dfx17,ivx17,cutx17,woex17 =mono_bin(data.y, data.alop_avg_amt_12m, n=6)#alop_avg_amt_12m-行用卡额度
dfx18,ivx18,cutx18,woex18 =mono_bin(data.y, data.consume_num_session12, n=6)#consume_num_session12-行用卡额度
dfx19,ivx19,cutx19,woex19 =mono_bin(data.y, data.consume_num_session6, n=6)#consume_num_session6-行用卡额度
dfx20,ivx20,cutx20,woex20 =mono_bin(data.y, data.consume_num_session3, n=6)#consume_num_session3-行用卡额度
#dfx21,ivx21,cutx21,woex21 =mono_bin(data.y, data.consume_num_session, n=6)#consume_num_session-行用卡额度
dfx22,ivx22,cutx22,woex22 =mono_bin(data.y, data.consume_amt_session12, n=6)#consume_amt_session12-行用卡额度
dfx23,ivx23,cutx23,woex23 =mono_bin(data.y, data.consume_amt_session6, n=6)#consume_amt_session6-行用卡额度
dfx24,ivx24,cutx24,woex24 =mono_bin(data.y, data.consume_amt_session3, n=6)#consume_amt_session3-行用卡额度
dfx25,ivx25,cutx25,woex25 =mono_bin(data.y, data.consume_amt_session, n=6)#consume_amt_session-行用卡额度
dfx26,ivx26,cutx26,woex26 =mono_bin(data.y, data.month_avg_use_year, n=6)#month_avg_use_year-行用卡额度
dfx27,ivx27,cutx27,woex27 =mono_bin(data.y, data.month_avg_use_month6, n=6)#month_avg_use_month6-行用卡额度
dfx28,ivx28,cutx28,woex28 =mono_bin(data.y, data.month_avg_use_month3, n=6)#month_avg_use_month3-行用卡额度


In [22]:
cutx1 = [ninf,0,1,2,3,4,pinf]
cutx2 = [ninf,0,1,2,3,4,pinf]
cutx3 = [ninf,0,1,2,3,4,pinf]
cutx4 = [ninf,0,1,2,3,4,pinf]
cutx5 = [ninf,0,1,2,3,4,pinf]
cutx6 = [ninf,0,1,2,3,4,pinf]
cutx7 = [ninf,0,1,2,3,4,pinf]

In [ ]:
特征标准化


In [ ]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
 
if __name__ == "__main__":
    # 读取葡萄酒数据集
    data = pd.read_csv("G:/dataset/wine.csv")
    # 获取第二列Alcohol
    x = data["Alcohol"]
    std = StandardScaler()
    #将x进行标准化
    x_std = std.fit_transform(x)
    print(x_std)
